#  Predicting the Solubility of Small Molecules using DeepChem
## Setup

The first step is to get DeepChem up and running. You'll also need to run the following commands to get DeepChem installed on your colab notebook.

In [1]:
!pip install --pre deepchem

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 608 kB 34.0 MB/s 
     |████████████████████████████████| 22.7 MB 65.4 MB/s 


We can now import the `deepchem` package to play with.

In [2]:
import deepchem as dc
dc.__version__

'2.6.1'

# Training a Model with DeepChem

Deep learning can be used to solve many sorts of problems, but the basic workflow is usually the same.  Here are the typical steps you follow.

1. Select the data set you will train your model on (or create a new data set if there isn't an existing suitable one).
2. Create the model.
3. Train the model on the data.
4. Evaluate the model on an independent test set to see how well it works.
5. Use the model to make predictions about new data.

With DeepChem, each of these steps can be as little as one or two lines of Python code.  In this tutorial we will walk through a basic example showing the complete workflow to solve a real world scientific problem.

The problem we will solve is predicting the solubility of small molecules given their chemical formulas.  This is a very important property in drug development: if a proposed drug isn't soluble enough, you probably won't be able to get enough into the patient's bloodstream to have a therapeutic effect.  The first thing we need is a data set of measured solubilities for real molecules.  One of the core components of DeepChem is MoleculeNet, a diverse collection of chemical and molecular data sets.  For this tutorial, we can use the Delaney solubility data set. The property of solubility in this data set is reported in log(solubility) where solubility is measured in moles/liter.

In [3]:
# The Delaney (ESOL) dataset a regression dataset containing structures and
# water solubility data for 1128 compounds. The dataset is widely used to
# validate machine learning models on estimating solubility directly from
# molecular structures (as encoded in SMILES strings).
# featurizer: the featurizer to use for processing the data.
tasks, datasets, transformers = dc.molnet.load_delaney(featurizer='GraphConv')
train_dataset, valid_dataset, test_dataset = datasets

In [12]:
print(test_dataset.ids)

['c1cc2ccc3cccc4ccc(c1)c2c34' 'Cc1cc(=O)[nH]c(=S)[nH]1'
 'Oc1ccc(cc1)C2(OC(=O)c3ccccc23)c4ccc(O)cc4 '
 'c1ccc2c(c1)cc3ccc4cccc5ccc2c3c45' 'C1=Cc2cccc3cccc1c23' 'CC1CO1'
 'CCN2c1ccccc1N(C)C(=S)c3cccnc23 ' 'CC12CCC3C(CCc4cc(O)ccc34)C2CCC1=O'
 'Cn2cc(c1ccccc1)c(=O)c(c2)c3cccc(c3)C(F)(F)F'
 'ClC(Cl)(Cl)C(NC=O)N1C=CN(C=C1)C(NC=O)C(Cl)(Cl)Cl '
 'COc2c1occc1cc3ccc(=O)oc23 ' 'CN2C(=C(O)c1ccccc1S2(=O)=O)C(=O)Nc3ccccn3 '
 'Cc3cc2nc1c(=O)[nH]c(=O)nc1n(CC(O)C(O)C(O)CO)c2cc3C'
 'c1ccc(cc1)c2ccc(cc2)c3ccccc3'
 'CC34CC(=O)C1C(CCC2=CC(=O)CCC12C)C3CCC4(=O) ' 'c1ccc2c(c1)sc3ccccc23'
 'CC23Cc1cnoc1C=C2CCC4C3CCC5(C)C4CCC5(O)C#C' 'OC(C(=O)c1ccccc1)c2ccccc2'
 'OCC2OC(Oc1ccccc1CO)C(O)C(O)C2O' 'CC3C2CCC1(C)C=CC(=O)C(=C1C2OC3=O)C'
 'O=Cc2ccc1OCOc1c2 ' 'CC1CCCCC1NC(=O)Nc2ccccc2'
 'CC(=O)N(S(=O)c1ccc(N)cc1)c2onc(C)c2C ' 'C1N(C(=O)NCC(C)C)C(=O)NC1'
 'CNC(=O)Oc1ccccc1C2OCCO2' 'CC1=C(CCCO1)C(=O)Nc2ccccc2 '
 'Cn2c(=O)on(c1ccc(Cl)c(Cl)c1)c2=O' 'C1Cc2cccc3cccc1c23'
 'c1ccc2cc3c4cccc5cccc(c3cc2c1)c45' 'Nc1cc(nc(N)n1=O)

First, notice the `featurizer` argument passed to the `load_delaney()` function.  Molecules can be represented in many ways.  We therefore tell it which representation we want to use, or in more technical language, how to "featurize" the data.  Second, notice that we actually get three different data sets: a training set, a validation set, and a test set.  Each of these serves a different function in the standard deep learning workflow.

Now that we have our data, the next step is to create a model.  We will use a particular kind of model called a "graph convolutional network", or "graphconv" for short.

In [4]:
# Build Graph Convolutional Models.
# These graph convolutions start with a per-atom set of
# descriptors for each atom in a molecule, then combine and recombine these
# descriptors over convolutional layers.
model = dc.models.GraphConvModel(n_tasks=1, mode='regression', dropout=0.2)

We now need to train the model on the data set.  We simply give it the data set and tell it how many epochs of training to perform (that is, how many complete passes through the data to make).

In [5]:
import warnings
warnings.filterwarnings('ignore')
# Returns the average loss over the most recent checkpoint interval
model.fit(train_dataset, nb_epoch=200)

0.06867386817932129

If everything has gone well, we should now have a fully trained model!  But do we?  To find out, we must evaluate the model on the test set.  We do that by selecting an evaluation metric and calling `evaluate()` on the model.  For this example, let's use the Pearson correlation, also known as r<sup>2</sup>, as our metric.  We can evaluate it on both the training set and test set.

In [6]:
metric = dc.metrics.Metric(dc.metrics.pearson_r2_score)
print("Training set score:", model.evaluate(train_dataset, [metric], transformers))
print("Test set score:", model.evaluate(test_dataset, [metric], transformers))

Training set score: {'pearson_r2_score': 0.9487058186764065}
Test set score: {'pearson_r2_score': 0.6659457210069061}


Notice that it has a higher score on the training set than the test set.  Models usually perform better on the particular data they were trained on than they do on similar but independent data.  This is called "overfitting", and it is the reason it is essential to evaluate your model on an independent test set.

Our model still has quite respectable performance on the test set.  For comparison, a model that produced totally random outputs would have a correlation of 0, while one that made perfect predictions would have a correlation of 1.  Our model does quite well, so now we can use it to make predictions about other molecules we care about.

Let's just use the first ten molecules from the test set.  For each one we print out the chemical structure (represented as a SMILES string) and the predicted log(solubility). To put these predictions in 
context, we print out the log(solubility) values from the test set as well.

In [7]:
solubilities = model.predict_on_batch(test_dataset.X[:10])
for molecule, solubility, test_solubility in zip(test_dataset.ids, solubilities, test_dataset.y):
    print(solubility, test_solubility, molecule)

[-1.852958] [-1.60114461] c1cc2ccc3cccc4ccc(c1)c2c34
[1.1366396] [0.20848251] Cc1cc(=O)[nH]c(=S)[nH]1
[-0.59835327] [-0.01602738] Oc1ccc(cc1)C2(OC(=O)c3ccccc23)c4ccc(O)cc4 
[-2.0032349] [-2.82191713] c1ccc2c(c1)cc3ccc4cccc5ccc2c3c45
[-1.601969] [-0.52891635] C1=Cc2cccc3cccc1c23
[1.9339883] [1.10168349] CC1CO1
[-0.16470852] [-0.88987406] CCN2c1ccccc1N(C)C(=S)c3cccnc23 
[-0.988208] [-0.52649706] CC12CCC3C(CCc4cc(O)ccc34)C2CCC1=O
[-0.70402926] [-0.76358725] Cn2cc(c1ccccc1)c(=O)c(c2)c3cccc(c3)C(F)(F)F
[0.2737535] [-0.64020358] ClC(Cl)(Cl)C(NC=O)N1C=CN(C=C1)C(NC=O)C(Cl)(Cl)Cl 
